In [1]:
import random


class CuatroEnLinea:
    def __init__(self, player1, player2,cant_filas,cant_columnas):

        self.player1, self.player2 = player1, player2
        self.player1_turn = random.choice([True, False])
        self.tablero = Tablero(cant_filas,cant_columnas)


    def play_game(self):
        #self.player1.start_game(1)
        #self.player2.start_game(2)

        terminamos = False

        #print self.player1.simbolo
        #print self.player2.simbolo
        
        while not terminamos:
            if self.player1_turn:
                player, simbolo = self.player1, self.player1.simbolo
            else:
                player, simbolo = self.player2, self.player2.simbolo

            self.mostrar_tablero()
            # print "le toca al jugador " + str(simbolo)
            
            jugada_valida = False
            while not jugada_valida:
                accion = player.move(self)
                jugada_valida = accion in self.tablero.acciones_posibles()
                if not jugada_valida:
                    print str(accion) + " imposible.. volve a intentar"
                
                
            self.tablero.poner_ficha(accion,simbolo)      

            if self.gana_jugador(simbolo):
                print("ganó el jugador " + str(simbolo))
                self.mostrar_tablero()
                terminamos = True
                return simbolo


            if self.hay_empate():
                #print("tablero lleno")
                terminamos = True
                return 0

            self.player1_turn = not self.player1_turn


            #if self.player_wins():
            #   player.reward(1, self.board)
            #  other_player.reward(-1, self.board)
            # break
            #if self.board_full(): # tie game
            #    player.reward(0.5, self.board)
            #    other_player.reward(0.5, self.board)
            #    break
            #other_player.reward(0, self.board)

    def gana_jugador(self,simbolo):
        return self.tablero.gana_jugador(simbolo)

    def mostrar_tablero(self):
        self.tablero.mostrar_tablero()

    def poner_ficha(self,columna,simbolo):
        self.tablero.poner_ficha(columna,simbolo)


    def hay_empate(self):
        return self.tablero.tablero_lleno()

    def dame_copia_del_tablero(self):
        return self.tablero.copy()


#########################################################################################################



class Tablero:

    
    def __init__(self, cant_filas,cant_columnas):

        self.cant_filas = cant_filas
        self.cant_columnas = cant_columnas
        self.tablero = [[0 for j in range(0,cant_columnas)] for i in range(0,cant_filas)]

    def __eq__(self,other):
        return self.tablero == other.tablero    
        
    def __hash__(self):
        res = []
        for i in self.tablero:
            res = res + i

        return hash(tuple(res))
        
    def copy(self):

        nuevo_tablero = Tablero(self.cant_filas,self.cant_columnas)
        nuevo_estado = nuevo_tablero.tablero

        for i in range(0,self.cant_filas):
            for j in range(0,self.cant_columnas):
                nuevo_estado[i][j] = self.tablero[i][j]

        return nuevo_tablero

    def tablero_lleno(self):
        acciones_posibles = self.acciones_posibles()
        return len(acciones_posibles) == 0

    def mostrar_tablero(self):
        return True
        #for i in reversed(range(0,self.cant_filas)):
            # print self.tablero[i]
            #print "\n"
            
    
    def gana_jugador(self,simbolo):
        tablero = self.tablero
        for i in reversed(range(0,self.cant_filas)):
            for j in range(0,self.cant_columnas):
                if tablero[i][j] == simbolo:
                    if self.cuatro_en_linea(i,j,i-3,j) or self.cuatro_en_linea(i,j,i-3,j+3) or self.cuatro_en_linea(i,j,i,j+3) or self.cuatro_en_linea(i,j,i+3,j+3) or self.cuatro_en_linea(i,j,i+3,j) or self.cuatro_en_linea(i,j,i+3,j-3) or self.cuatro_en_linea(i,j,i,j-3) or self.cuatro_en_linea(i,j,i-3,j-3):
                        return True
        return False


    def hay_empate(self):
        return self.tablero_lleno() and not self.gana_jugador(1) and not self.gana_jugador(2)

    def build_range(self,i1,i2):

        rango = []
        if ( i1 < i2 ):
            rango = [i1,i1+1,i1+2,i1+3]
        elif ( i1 == i2 ):
            rango = [i1,i1,i1,i1]
        else:
            rango = [i1,i1-1,i1-2,i1-3]

        return rango


    def fila_disponible_en(self,j):
        for i in range(0,self.cant_filas):
            if self.tablero[i][j] == 0:
                return i
        return -1


    def poner_ficha(self,j,simbolo):
        tablero = self.tablero
        columna = j
        for i in range(0,self.cant_filas):
            if tablero[i][columna] == 0:
                tablero[i][columna] = simbolo
                break
                
    def cuatro_en_linea(self, i1,j1,i2,j2):

        tablero = self.tablero
        
        if i2 < 0 or i2 >= self.cant_filas or j2 < 0 or j2 >= self.cant_columnas:
            return False

        fila = self.build_range(i1,i2)
        columna = self.build_range(j1,j2)

        return (tablero[fila[0]][columna[0]] == tablero[fila[1]][columna[1]] == tablero[fila[2]][columna[2]] == tablero[fila[3]][columna[3]]) and tablero[fila[0]][columna[0]] != 0


    def proximo_movimiento_hay_ganador(self,simbolo):
        
        for accion in self.acciones_posibles():
            proximo_estado = self.proximo_estado_para(accion,simbolo)
            if self.gana_jugador(simbolo):
                return True
        
        return False    

    def acciones_posibles(self):
        accionesPosibles = []
        tablero = self.tablero
        for j in range(0,self.cant_columnas):
            termine_columna = False
            for i in range(0,self.cant_filas):
                if tablero[i][j] == 0 and not termine_columna:
                    accionesPosibles.append(j)
                    termine_columna = True


        return accionesPosibles


    def proximo_estado_para(self,accion,simbolo):
        nuevo_tablero = self.copy()
        nuevo_tablero.poner_ficha(accion,simbolo)

        return nuevo_tablero



#########################################################################################################


class Player():
    def __init__(self,simbolo):
        self.simbolo = simbolo
        self.breed = "human"

    def move(self,tablero):
        accion = int(raw_input("Qué columna elegís? "))
        
        return accion - 1


class RandomPLayer():
    def __init__(self,simbolo):
        self.breed = "random"
        self.simbolo = simbolo

    def move(self,tablero):
        return random.randint(0,6)

#########################################################################################################



class QPlayer():
    def __init__(self,simbolo,alpha,gamma,profundidad):
        self.breed = "qplayer"
        self.gamma = gamma
        self.alpha = alpha
        self.tabla_q = {}
        self.simbolo = simbolo
        self.profundidad = profundidad

    def move(self,juego):
        estado = juego.dame_copia_del_tablero()
        accion = self.mejor_accion_para(estado, self.profundidad)
        return accion

    def mejor_accion_para(self,estado,profundidad):
        
        acciones_posibles = estado.acciones_posibles()
        
        if random.random() < 0.2:
            
            jugada_random = random.randint(0,len(acciones_posibles)-1)
            mejor_accion = acciones_posibles[jugada_random]
                
        
        else:
            mejor_valor = -100
            mejor_accion = acciones_posibles[0]
            valor_candidato = 0
        
            for j in acciones_posibles:
                valor_candidato = self.q(estado,j,profundidad)
                if mejor_valor < valor_candidato:
                    mejor_valor = valor_candidato
                    mejor_accion = j

            viejo_valor = self.obtener_valor_en_tabla_q(estado,mejor_accion)
            self.tabla_q[(estado,mejor_accion)] = viejo_valor + self.alpha * ( self.recompensa(estado,mejor_accion) + self.gamma * mejor_valor - viejo_valor )
        #print "La recompensa es"
        #print self.recompensa(estado,mejor_accion)

        return mejor_accion
    
    def simbolo_del_rival(self):
        if self.simbolo == 1:
            return 2
        else:
            return 1
    
    def dos_en_linea_y_prox_libre(self,simbolo,i,j,estado):
        cantidad_filas = estado.cant_filas
        cantidad_col = estado.cant_columnas
        
        estado_ = estado.copy()
        tablero_del_estado = estado_.tablero
        dos_en_linea = False
        if  i > cantidad_filas or i < 0 or j > cantidad_col or j<0:
            return False
        elif 0 <= j-2 and j+1 < cantidad_col:
            if tablero_del_estado[i][j-2]==tablero_del_estado[i][j-1]== self.simbolo and tablero_del_estado[i][j+1]==0:
                return True
            elif 0<= i-2 and i+1 < cantidad_filas:
                if (tablero_del_estado[i-2][j-2]==tablero_del_estado[i-1][j-1]== self.simbolo and tablero_del_estado[i+1][j+1]==0) or (tablero_del_estado[i+1][j+1]==tablero_del_estado[i-1][j-1]== self.simbolo and tablero_del_estado[i-2][j-2]==0):
                    return True
        elif j+2 < cantidad_col and 0<= j-1:
            if (tablero_del_estado[i][j+2]==tablero_del_estado[i][j+1]== self.simbolo and tablero_del_estado[i][j-1]==0):
                return True
            elif 0<= i-2 and i+1 < cantidad_filas:
                if (tablero_del_estado[i-1][j+1]==tablero_del_estado[i-2][j+2]== self.simbolo and tablero_del_estado[i+1][j-1]==0) or (tablero_del_estado[i-1][j+1]==tablero_del_estado[i+1][j-1]== self.simbolo and tablero_del_estado[i-2][j+2]==0):
                    return True
        elif 0 <= i-2 :
                if tablero_del_estado[i-1][j]==tablero_del_estado[i-2][j]== self.simbolo:
                    return True
        else:
                return False

        
            
            
    def q(self,estado,accion,profundidad):

        peor_valor = -100

        proximo_estado = estado.proximo_estado_para(accion,self.simbolo)

        
        if proximo_estado.gana_jugador(self.simbolo):
            peor_valor = 100
        elif proximo_estado.proximo_movimiento_hay_ganador(self.simbolo_del_rival()):
            peor_valor = -100
        elif proximo_estado.hay_empate():
            peor_valor = 0
        elif profundidad == 0:
            #print "prof 0"
            #print accion

            acciones_posibles_del_rival = proximo_estado.acciones_posibles()

            for j in acciones_posibles_del_rival:

                proximo_estado_a_explorar = proximo_estado.proximo_estado_para(j,self.simbolo)
                nuevas_acciones_posibles_mias = proximo_estado_a_explorar.acciones_posibles()

                for k in nuevas_acciones_posibles_mias:
                    valor_en_tabla_q = self.obtener_valor_en_tabla_q(proximo_estado_a_explorar,k)
                    if peor_valor > valor_en_tabla_q:
                        peor_valor = valor_en_tabla_q
        else:
            acciones_posibles_del_rival = proximo_estado.acciones_posibles()

            for j in acciones_posibles_del_rival:

                proximo_estado_a_explorar = proximo_estado.proximo_estado_para(j,self.simbolo)
                nuevas_acciones_posibles_mias = proximo_estado_a_explorar.acciones_posibles()

                for k in nuevas_acciones_posibles_mias:
                    #print profundidad
                    valor_en_tabla_q = self.q(proximo_estado_a_explorar,k,profundidad-1)
                    if peor_valor > valor_en_tabla_q:
                        peor_valor = valor_en_tabla_q


        return peor_valor


    def recompensa1(self,estado,accion):
        return 0
    
    
    def recompensa(self,estado,accion):
        tablero = estado.tablero
        recompensa_max = 100
        recompensa_media=30
        recompensa_baja =10
        recompensa_cero=0
        anti_recompensa = -100
        j =accion

        
        proximo_estado = estado.proximo_estado_para(j,self.simbolo)

        
        if proximo_estado.gana_jugador(self.simbolo):
            return recompensa_max
        elif proximo_estado.proximo_movimiento_hay_ganador(self.simbolo_del_rival()):
            return anti_recompensa 
        elif proximo_estado.hay_empate():
            return  recompensa_cero 
         #Busco la fila donde quedara la ficha una vez que juegue en la columna "j"
        cantidad_filas= estado.cant_filas
        termine = False
        
        for i in range(0,cantidad_filas):
            if (tablero[i][j]==0 and termine==False):
                fila_mejor_accion = i
                termine = True
                if self.dos_en_linea_y_prox_libre(self.simbolo,i,j,estado):
                    return recompensa_media
                elif self.dos_en_linea_y_prox_libre(self.simbolo_del_rival(),i,j,estado):
                    recompensa_baja
                #elif if hay_mio_al_lado:
                    #return recompensa_baja
                else:
                    return recompensa_cero

    def recompensa_por_defecto(self):
        return 1


    def obtener_valor_en_tabla_q(self,estado,accion):
        if (estado,accion) in self.tabla_q:
            return self.tabla_q.get((estado,accion))
        else:
            return 1




#########################################################################################################



In [39]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [0,0.25,0.5,0.75,1]
gammas = [0,0.25,0.5,0.75,1]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,100):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            print cantidad_ganadas_p1
            print cantidad_ganadas_p2



print valores   
    



{}
ganó el jugador 1
1
1
0
ganó el jugador 1
2
2
0
ganó el jugador 1
3
3
0
ganó el jugador 1
4
4
0
ganó el jugador 1
5
5
0
ganó el jugador 2
6
5
1
ganó el jugador 1
7
6
1
ganó el jugador 1
8
7
1
ganó el jugador 1
9
8
1
ganó el jugador 1
10
9
1
ganó el jugador 2
11
9
2
ganó el jugador 1
12
10
2
ganó el jugador 1
13
11
2
ganó el jugador 1
14
12
2
ganó el jugador 1
15
13
2
ganó el jugador 1
16
14
2
ganó el jugador 1
17
15
2
ganó el jugador 2
18
15
3
ganó el jugador 2
19
15
4
ganó el jugador 1
20
16
4
ganó el jugador 1
21
17
4
ganó el jugador 1
22
18
4
ganó el jugador 1
23
19
4
ganó el jugador 2
24
19
5
ganó el jugador 2
25
19
6
0 imposible.. volve a intentar
ganó el jugador 2
26
19
7
ganó el jugador 1
27
20
7
ganó el jugador 2
28
20
8
ganó el jugador 2
29
20
9
ganó el jugador 1
30
21
9
ganó el jugador 1
31
22
9
ganó el jugador 1
32
23
9
ganó el jugador 1
33
24
9
ganó el jugador 1
34
25
9
ganó el jugador 1
35
26
9
ganó el jugador 1
36
27
9
ganó el jugador 1
37
28
9
ganó el jugador 2
38
28


In [33]:
a ={}

a[(1,2)] = 5

print a

{(1, 2): 5}


In [40]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas1 = [0,0.25,0.5,0.75,1]
gammas1 = [0,0.25,0.5,0.75,1]

alphas = [0,1]
gammas = [0,1]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,500):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            print cantidad_ganadas_p1
            print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 1
1
1
0
ganó el jugador 2
2
1
1
ganó el jugador 2
3
1
2
ganó el jugador 2
4
1
3
ganó el jugador 1
5
2
3
0 imposible.. volve a intentar
ganó el jugador 2
6
2
4
ganó el jugador 2
7
2
5
ganó el jugador 1
8
3
5
ganó el jugador 1
9
4
5
ganó el jugador 1
10
5
5
ganó el jugador 1
11
6
5
ganó el jugador 2
12
6
6
ganó el jugador 1
13
7
6
ganó el jugador 1
14
8
6
ganó el jugador 1
15
9
6
ganó el jugador 1
16
10
6
2 imposible.. volve a intentar
2 imposible.. volve a intentar
ganó el jugador 1
17
11
6
ganó el jugador 1
18
12
6
1 imposible.. volve a intentar
1 imposible.. volve a intentar
0 imposible.. volve a intentar
2 imposible.. volve a intentar
ganó el jugador 2
19
12
7
ganó el jugador 1
20
13
7
ganó el jugador 1
21
14
7
ganó el jugador 1
22
15
7
ganó el jugador 1
23
16
7
ganó el jugador 1
24
17
7
ganó el jugador 2
25
17
8
0 imposible.. volve a intentar
ganó el jugador 1
26
18
8
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 1
27
19
8
0 imposib

Algunos resultados:
Diccionarios {alpha, gamma: cantidad de ganadas por Qplayer}
1)  epsilon 0.2
    alphas = [0,0.25,0.5,0.75,1]
    gammas = [0,0.25,0.5,0.75,1]
    Tirando 100 corridas, obtuvimos
    {(0, 0.5): 79, (0.5, 1): 78, (0.75, 0): 72, (0.25, 0.25): 78, (1, 0.5): 85, (0.5, 0.5): 70, (0.75, 0.75): 81, (1, 0.25): 78, (0.75, 0.5): 82, (1, 1): 70, (0.25, 1): 75, (0.25, 0): 76, (0.5, 0): 77, (0, 0.25): 76, (0, 0.75): 82, (0.25, 0.75): 76, (1, 0): 73, (0.5, 0.25): 69, (1, 0.75): 76, (0, 1): 82, (0, 0): 78, (0.75, 1): 78, (0.5, 0.75): 73, (0.75, 0.25): 80, (0.25, 0.5): 82}

2)  epsilon 0.2
    alphas = [0,1]
    gammas = [0,1]
   Tirando 500 corridas, obtuvimos:
{(0, 1): 387, (1, 0): 391, (0, 0): 364, (1, 1): 393}
3)  epsilon 0.2
    alphas = 
    gammas = 
    Tirando 200 corridas obtuvimos:
    
    
    
    FALTAN LOS CASOS (0.5,0) (0.25,0.75) (0.25, 1) (0.75,1) (1,1)

In [42]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [0.25,0.5,0.75,1]
gammas = [0,0.25,0.5,0.75,1]

#alphas = [1]
#gammas = [0.25]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,200):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            #print cantidad_ganadas_p1
            #print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 1
1
ganó el jugador 2
2
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 1
3
ganó el jugador 2
4
ganó el jugador 1
5
ganó el jugador 1
6
ganó el jugador 1
7
0 imposible.. volve a intentar
ganó el jugador 1
8
ganó el jugador 1
9
ganó el jugador 1
10
ganó el jugador 1
11
ganó el jugador 1
12
ganó el jugador 1
13
ganó el jugador 2
14
ganó el jugador 1
15
ganó el jugador 1
16
0 imposible.. volve a intentar
ganó el jugador 1
17
ganó el jugador 1
18
0 imposible.. volve a intentar
ganó el jugador 1
19
0 imposible.. volve a intentar
0 imposible.. volve a intentar
0 imposible.. volve a intentar
1 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
20
ganó el jugador 1
21
ganó el jugador 1
22
0 imposible.. volve a intentar
ganó el jugador 1
23
ganó el jugador 1
24
ganó el jugador 1
25
ganó el jugador 2
26
ganó el jugador 1
27
ganó el jugador 1
28
ganó el jugador 1
29
ganó el jugador 1
30
ganó el jugador 2
31
ganó el jugado

KeyboardInterrupt: 

In [2]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [1]
gammas = [0.5]

#alphas = [1]
#gammas = [0.25]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,501):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            #print cantidad_ganadas_p1
            #print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 2
1
ganó el jugador 1
2
ganó el jugador 1
3
ganó el jugador 2
4
ganó el jugador 1
5
ganó el jugador 1
6
0 imposible.. volve a intentar
ganó el jugador 2
7
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
8
ganó el jugador 1
9
ganó el jugador 1
10
ganó el jugador 1
11
ganó el jugador 1
12
ganó el jugador 1
13
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
14
ganó el jugador 1
15
0 imposible.. volve a intentar
ganó el jugador 1
16
ganó el jugador 2
17
ganó el jugador 1
18
ganó el jugador 1
19
ganó el jugador 1
20
0 imposible.. volve a intentar
ganó el jugador 1
21
ganó el jugador 2
22
ganó el jugador 1
23
ganó el jugador 1
24
ganó el jugador 1
25
ganó el jugador 1
26
ganó el jugador 1
27
ganó el jugador 1
28
ganó el jugador 2
29
ganó el jugador 1
30
ganó el jugador 1
31
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 1
32
ganó el jugador 1
33
ganó el jugador 1
34
ganó el jugad

In [3]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [1]
gammas = [0.5]

#alphas = [1]
#gammas = [0.25]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,501):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            #print cantidad_ganadas_p1
            #print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 1
1
ganó el jugador 1
2
ganó el jugador 1
3
ganó el jugador 1
4
ganó el jugador 1
5
ganó el jugador 1
6
ganó el jugador 1
7
ganó el jugador 1
8
ganó el jugador 1
9
0 imposible.. volve a intentar
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 1
10
ganó el jugador 1
11
ganó el jugador 1
12
ganó el jugador 1
13
ganó el jugador 1
14
ganó el jugador 1
15
ganó el jugador 1
16
ganó el jugador 1
17
ganó el jugador 2
18
ganó el jugador 1
19
ganó el jugador 1
20
ganó el jugador 1
21
ganó el jugador 1
22
0 imposible.. volve a intentar
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 1
23
ganó el jugador 2
24
ganó el jugador 1
25
ganó el jugador 1
26
ganó el jugador 1
27
ganó el jugador 1
28
ganó el jugador 1
29
ganó el jugador 1
30
0 imposible.. volve a intentar
ganó el jugador 1
31
ganó el jugador 1
32
ganó el jugador 1
33
ganó el jugador 1
34
ganó el jugador 1
35
ganó el jugador 1
36
ganó el jugador 1
37
ganó el juga

In [10]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [1]
gammas = [0.5]

#alphas = [1]
#gammas = [0.25]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,201):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            #print cantidad_ganadas_p1
            #print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 1
1
ganó el jugador 2
2
0 imposible.. volve a intentar
0 imposible.. volve a intentar
0 imposible.. volve a intentar
2 imposible.. volve a intentar
ganó el jugador 1
3
ganó el jugador 1
4
ganó el jugador 1
5
0 imposible.. volve a intentar
ganó el jugador 2
6
0 imposible.. volve a intentar
ganó el jugador 2
7
ganó el jugador 1
8
ganó el jugador 2
9
0 imposible.. volve a intentar
ganó el jugador 2
10
ganó el jugador 1
11
ganó el jugador 1
12
ganó el jugador 1
13
ganó el jugador 1
14
ganó el jugador 2
15
ganó el jugador 1
16
ganó el jugador 1
17
ganó el jugador 1
18
ganó el jugador 2
19
ganó el jugador 1
20
ganó el jugador 1
21
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
22
ganó el jugador 1
23
ganó el jugador 1
24
ganó el jugador 1
25
ganó el jugador 1
26
0 imposible.. volve a intentar
ganó el jugador 2
27
ganó el jugador 1
28
ganó el jugador 1
29
0 imposible.. volve a intentar
ganó el jugador 1
30
ganó el jugador 1
31
ganó el jugado

In [25]:
p2 = RandomPLayer(2)
#p2 =QPlayer(2,1,1,1)

cantidad_ganadas_p1=0
cantidad_ganadas_p2=0

alphas = [1]
gammas = [0.5]

#alphas = [1]
#gammas = [0.25]

valores = {}

for unGamma in gammas:
    for unAlpha in alphas:
        p1=QPlayer(1,unAlpha,unGamma,2)
        #p2 =QPlayer(2,unAlpha,unGamma,1)
        cantidad_ganadas_p1 = 0
        cantidad_ganadas_p2 = 0
        print valores
        for i in range (1,41):
            t = CuatroEnLinea(p1, p2,6,7)
            ganador = t.play_game()
            print i
            if ganador == p1.simbolo:
                cantidad_ganadas_p1=cantidad_ganadas_p1+1
            elif ganador == p2.simbolo:
                cantidad_ganadas_p2=cantidad_ganadas_p2 + 1
                
            valores[(unAlpha,unGamma)] = cantidad_ganadas_p1
    
            #print cantidad_ganadas_p1
            #print cantidad_ganadas_p2



print valores 

{}
ganó el jugador 1
1
ganó el jugador 1
2
ganó el jugador 2
3
ganó el jugador 1
4
ganó el jugador 1
5
ganó el jugador 1
6
ganó el jugador 1
7
ganó el jugador 1
8
ganó el jugador 1
9
ganó el jugador 1
10
ganó el jugador 1
11
ganó el jugador 1
12
ganó el jugador 1
13
ganó el jugador 1
14
ganó el jugador 2
15
ganó el jugador 1
16
ganó el jugador 2
17
ganó el jugador 1
18
ganó el jugador 1
19
ganó el jugador 1
20
0 imposible.. volve a intentar
ganó el jugador 2
21
0 imposible.. volve a intentar
ganó el jugador 1
22
ganó el jugador 1
23
ganó el jugador 1
24
ganó el jugador 1
25
ganó el jugador 1
26
ganó el jugador 1
27
ganó el jugador 1
28
1 imposible.. volve a intentar
1 imposible.. volve a intentar
1 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
29
ganó el jugador 1
30
ganó el jugador 1
31
0 imposible.. volve a intentar
0 imposible.. volve a intentar
ganó el jugador 2
32
ganó el jugador 1
33
ganó el jugador 2
34
ganó el jugador 1
35
ganó el jugador 1
36
ga